In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.optimizers import Adam

import seaborn as sns
sns.set()

Using TensorFlow backend.


In [2]:
data = pd.read_csv('D:/IITM/DeepCache/Medysin/mediSynDataset_x2_O3488.csv')
data.head()

,object_ID,request_time
0,2033,86416.158584
1,1280,86419.381316
2,3482,86426.764935
3,2764,86429.048406
4,2698,86430.680023


In [3]:
data = np.array(data)
data.shape

(218855802, 2)

In [4]:
np.min(data[:,0]),np.max(data[:,0])

(1.0, 3488.0)

In [5]:
np.min(data[:,1]),np.max(data[:,1])

(86416.15858448479, 17343055.15509677)

In [52]:
data[:,1] = data[:,1] - np.min(data[:,1])

In [66]:
#to find requests in 1h
i = 2000
while data[i,1] < 3600:
    i += 1
print(i,data[i,1])

2281 3603.9286053070246


In [67]:
# to find the probability of occurance of Oi object is caculated as Ni/1000
# where Ni is number of occurance of Oi in past 1k req

count = np.zeros(3488)
for i in range(2281):
    for j in range(3488):
        if data[i,0] == j+1:
            count[j] += 1

In [82]:
interval = data[2281,1] 
j = 0
for i in range(2281,4000):
    interval = interval + data[i,1] - data[i-1,1]
    while interval > 3600:
        j +=1
        interval = interval - data[j,1]
        
    print(j,i)
    print(data[i,1] - data[j,1])

2 2281
3593.3222544163145
2 2282
3593.4459508657746
3 2283
3600.6403630157147
3 2284
3602.5821286877763
3 2285
3602.7614435842115
3 2286
3603.1773753792804
3 2287
3603.4989454900788
3 2288
3605.5316864872584
3 2289
3605.5466888804804
3 2290
3606.256939841769
3 2291
3606.5810177374224
3 2292
3607.059825019678
3 2293
3609.7977660062606
4 2294
3612.0256396941113
4 2295
3612.7438224612124
4 2296
3617.008636490442
4 2297
3617.6738850240654
4 2298
3618.9838428005605
4 2299
3619.400072484408
4 2300
3619.945275647129
4 2301
3620.82619139539
4 2302
3621.9126115754625
5 2303
3615.60691664755
5 2304
3615.6925828771055
5 2305
3616.9948754327925
5 2306
3619.064842401858
5 2307
3619.8004345045483
5 2308
3620.62891960502
5 2309
3621.751681234484
5 2310
3624.5424782004266
5 2311
3625.0759488600015
5 2312
3626.04687685486
5 2313
3626.4902082707995
5 2314
3627.5830372566707
5 2315
3628.549163671458
5 2316
3629.571066287899
5 2317
3631.732698722335
5 2318
3636.8374750912335
6 2319
3636.7354559064115
6 23

4416.346921776567
24 2720
4419.74898386604
24 2721
4421.6542869882105
24 2722
4426.834509981796
24 2723
4427.105415056096
24 2724
4427.473418989146
24 2725
4429.516464731627
24 2726
4431.264129341624
24 2727
4431.266754566197
24 2728
4436.0056351598905
24 2729
4436.037451051598
24 2730
4437.485891204909
24 2731
4439.952869453933
25 2732
4442.021492671571
25 2733
4442.887866430436
25 2734
4444.8265557459235
25 2735
4448.46272865287
25 2736
4448.685895629824
25 2737
4448.807085949244
25 2738
4451.278366378916
25 2739
4452.987893925485
25 2740
4453.172239617183
25 2741
4455.109893587272
25 2742
4456.621237007421
25 2743
4459.467948471007
25 2744
4461.2260377656785
25 2745
4463.151072654626
25 2746
4463.924434547123
25 2747
4471.080288656376
25 2748
4472.973396256042
25 2749
4475.256134794152
25 2750
4475.924904119107
25 2751
4480.058574489565
25 2752
4480.549913879382
25 2753
4481.01136505224
25 2754
4481.8209364504
25 2755
4482.2799255344435
25 2756
4485.440909761848
25 2757
4487.3073377

35 3053
5117.387773112074
35 3054
5119.2487909969495
35 3055
5120.148278400709
35 3056
5123.927148698916
35 3057
5124.221667280479
35 3058
5128.109910763582
35 3059
5129.636664024743
35 3060
5129.775192736954
35 3061
5130.22024072631
35 3062
5131.207771775546
35 3063
5135.662558925353
35 3064
5135.94514268871
36 3065
5138.195008653973
36 3066
5138.9121773394
36 3067
5139.993363646747
36 3068
5141.174280364561
36 3069
5141.39999244854
36 3070
5142.486114992935
36 3071
5144.072717097035
36 3072
5145.85920064972
36 3073
5146.435807917922
36 3074
5147.762360794397
36 3075
5149.908837408075
36 3076
5152.078153395065
36 3077
5153.421528303035
36 3078
5156.182294330472
36 3079
5160.432162864672
36 3080
5161.165155893774
36 3081
5161.211838313349
36 3082
5165.25301380569
36 3083
5165.609530249145
36 3084
5165.864906420204
36 3085
5167.089591985554
36 3086
5171.428262207366
36 3087
5173.969890269931
36 3088
5176.260973998447
36 3089
5176.2903760869085
36 3090
5178.770926457073
36 3091
5183.1130

45 3386
5802.50007364573
45 3387
5805.77794022701
45 3388
5806.052522036494
45 3389
5807.711455225784
45 3390
5810.987666698027
45 3391
5812.399339707466
45 3392
5816.363894092734
45 3393
5817.91799077444
45 3394
5818.154057706808
45 3395
5818.774037109848
45 3396
5818.797359695338
45 3397
5828.31536922924
45 3398
5829.390283409855
45 3399
5829.886572417992
45 3400
5832.1350634783885
45 3401
5834.557951186493
45 3402
5836.008703754647
45 3403
5837.759312857132
45 3404
5838.623391587913
45 3405
5839.466014197649
45 3406
5841.227327069471
45 3407
5841.745127827831
45 3408
5842.1782390177395
45 3409
5847.4071357592475
45 3410
5850.097602316222
45 3411
5852.044593311119
45 3412
5855.113114088104
45 3413
5855.856918731137
45 3414
5856.496708930441
45 3415
5856.827774471312
45 3416
5859.340831808338
45 3417
5860.836391438832
45 3418
5864.38359083797
45 3419
5871.835698338502
45 3420
5872.307876119245
45 3421
5873.544532609812
46 3422
5874.056319609081
46 3423
5874.858645310742
46 3424
5875.0

6904.558814471529
57 3886
6906.768462944674
57 3887
6906.847735086674
57 3888
6912.416100340342
57 3889
6916.026718343783
57 3890
6916.444338127607
57 3891
6916.7780115712085
57 3892
6917.607828575317
57 3893
6920.4773126421205
57 3894
6922.480153043376
57 3895
6923.448747413335
57 3896
6928.343099800695
57 3897
6928.3892410673725
57 3898
6935.482162486529
57 3899
6937.2432129119115
57 3900
6938.604953089598
57 3901
6938.724171603462
57 3902
6939.870348688331
57 3903
6940.9697654177435
57 3904
6943.592434582883
57 3905
6944.632036994342
57 3906
6945.822273820915
58 3907
6948.795279513855
58 3908
6950.159151443411
58 3909
6952.523610121905
58 3910
6956.504264398274
58 3911
6956.587482011848
58 3912
6958.680776741807
58 3913
6961.865223500747
58 3914
6961.923319657668
58 3915
6962.310032073423
58 3916
6963.338167683149
58 3917
6964.053110824025
58 3918
6964.754532028543
58 3919
6968.3404277371155
58 3920
6972.90683791069
58 3921
6973.2752716734685
58 3922
6975.943993825946
58 3923
6979.7

In [74]:
data[2601,1]-data[320,1]

3725.4080385281413